# SparkSQL Explain Plan

* [Spark’s Logical and Physical plans … When, Why, How and Beyond](https://medium.com/datalex/sparks-logical-and-physical-plans-when-why-how-and-beyond-8cd1947b605a)
* [SparkSQL - Explain Statement](https://spark.apache.org/docs/latest/sql-ref-syntax-qry-explain.html)

> The EXPLAIN statement is used to provide logical/physical plans for an input statement. By default, this clause provides information about a physical plan only.

* [pyspark.sql.DataFrame.explain(extended=None, mode=None)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.explain.html)

> #### extendedbool, optional
> default False. If False, prints only the physical plan. When this is a string without specifying the mode, it works as the mode is specified.
> 
> #### modestr, optional
> specifies the expected output format of plans.
> * simple: Print only a physical plan.
> * extended: Print both logical and physical plans.
> * codegen: Print a physical plan and generated codes if they are available.
> * cost: Print a logical plan and statistics if they are available.
> * formatted: Split explain output into two sections: a physical plan outline and node details.

In [2]:
%%html
<style>
table {float:left}
</style>

In [3]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

In [4]:
import os
import sys
import gc
from datetime import (
    datetime,
    date
)
import numpy as np
import pandas as pd

#  Environemnt Variables

## Hadoop

In [5]:
os.environ['HADOOP_CONF_DIR'] = "/opt/hadoop/hadoop-3.2.2/etc/hadoop"

In [6]:
%%bash
export HADOOP_CONF_DIR="/opt/hadoop/hadoop-3.2.2/etc/hadoop"
ls $HADOOP_CONF_DIR | head -n 5

capacity-scheduler.xml
configuration.xsl
container-executor.cfg
core-site.xml
core-site.xml.48132.2022-02-15@12:29:41~


## PYTHONPATH

Refer to the **pyspark** modules to load from the ```$SPARK_HOME/python/lib``` in the Spark installation.

* [PySpark Getting Started](https://spark.apache.org/docs/latest/api/python/getting_started/install.html)

> Ensure the SPARK_HOME environment variable points to the directory where the tar file has been extracted. Update PYTHONPATH environment variable such that it can find the PySpark and Py4J under SPARK_HOME/python/lib. One example of doing this is shown below:

```
export PYTHONPATH=$(ZIPS=("$SPARK_HOME"/python/lib/*.zip); IFS=:; echo "${ZIPS[*]}"):$PYTHONPATH
```

Alternatively install **pyspark** with pip or conda locally which installs the Spark runtime libararies (for standalone).

* [Can PySpark work without Spark?](https://stackoverflow.com/questions/51728177/can-pyspark-work-without-spark)

> As of v2.2, executing pip install pyspark will install Spark. If you're going to use Pyspark it's clearly the simplest way to get started. On my system Spark is installed inside my virtual environment (miniconda) at lib/python3.6/site-packages/pyspark/jars  
> PySpark has a Spark installation installed. If installed through pip3, you can find it with pip3 show pyspark. Ex. for me it is at ~/.local/lib/python3.8/site-packages/pyspark. This is a standalone configuration so it can't be used for managing clusters like a full Spark installation.

In [7]:
# os.environ['PYTHONPATH'] = "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip:/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
sys.path.extend([
    "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip",
    "/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
])

## PySpark package imports

Execute after the PYTHONPATH setup.

In [8]:
import pyspark.sql 
from pyspark.sql.types import *
from pyspark.sql.functions import (
    col,
    lit,
    avg,
    stddev,
    isnan,
    date_format,
    to_date,
    months_between,
    add_months,
)

---
# Spark Session


In [9]:
from pyspark.sql import SparkSession

In [10]:
spark = SparkSession.builder\
    .master('yarn') \
    .config('spark.submit.deployMode', 'client') \
    .config('spark.debug.maxToStringFields', 100) \
    .config('spark.executor.memory', '2g') \
    .getOrCreate()

2022-02-22 15:59:34,169 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-02-22 15:59:36,525 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-02-22 15:59:38,903 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [11]:
NUM_CORES = 4
NUM_PARTITIONS = 3

spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set('spark.sql.legacy.timeParserPolicy', 'LEGACY')

# DataFrame from Python data

* [SparkSession.createDataFrame(data, schema=None, samplingRatio=None, verifySchema=True)](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.SparkSession.createDataFrame.html)

# Schema Definition

* [Data Types](https://spark.apache.org/docs/latest/sql-ref-datatypes.html#data-types)

```from pyspark.sql.types import *```

| Data type | Value type in Python | API to access or create a data type |  |
|:---|:---|:---|:--|
|ByteType | int or long Note: Numbers will be converted to 1-byte signed integer numbers at runtime. Please make sure that numbers are within the range of -128 to 127. | ByteType() |  |
| ShortType | int or long Note: Numbers will be converted to 2-byte signed integer numbers at runtime. Please make sure that numbers are within the range of -32768 to 32767. | ShortType() |  |
| IntegerType | int or long | IntegerType() |  |
| LongType | long Note: Numbers will be converted to 8-byte signed integer numbers at runtime. Please make sure that numbers are within the range of -9223372036854775808 to 9223372036854775807.Otherwise, please convert data to decimal.Decimal and use DecimalType. | LongType() |  |
| FloatType | float Note: Numbers will be converted to 4-byte single-precision floating point numbers at runtime. | FloatType() |  |
| DoubleType | float | DoubleType() |  |
| DecimalType | decimal.Decimal | DecimalType() |  |
| StringType | string | StringType() |  |
| BinaryType | bytearray | BinaryType() |  |
| BooleanType | bool | BooleanType() |  |
| TimestampType | datetime.datetime | TimestampType() |  |
| DateType | datetime.date | DateType() |  |
| ArrayType | list, tuple, or array | ArrayType(elementType, [containsNull]) Note:The default value of containsNull is True. |  |
| MapType | dict | MapType(keyType, valueType, [valueContainsNull]) Note:The default value of valueContainsNull is True. |  |
| StructType | list or tuple | StructType(fields) Note: fields is a Seq of StructFields. Also, two fields with the same name are not allowed. |  |
| StructField | The value type in Python of the data type of this field (For example, Int for a StructField with the data type IntegerType) | StructField(name, dataType, [nullable]) Note: The default value of nullable is True. |  |


# Data


In [12]:
%%bash
hdfs dfs -mkdir -p flight
hdfs dfs -put -f ./data/flight/*.csv flight/

In [13]:
flight = spark.read\
    .option("compression", "none")\
    .option("header", True)\
    .option("sep", ',')\
    .option("nullValue", np.nan)\
    .option("inferSchema", True)\
    .option("dateFormat", "yyyy-MM-dd")\
    .csv("flight/flightData.csv")\
    .withColumn("date", to_date(col("date"), "yyyy-MM-dd"))

flight.printSchema()
flight.createOrReplaceTempView("flight")

root
 |-- passengerId: integer (nullable = true)
 |-- flightId: integer (nullable = true)
 |-- from: string (nullable = true)
 |-- to: string (nullable = true)
 |-- date: date (nullable = true)



In [14]:
flight.show(truncate=False)

+-----------+--------+----+---+----------+
|passengerId|flightId|from|to |date      |
+-----------+--------+----+---+----------+
|48         |0       |cg  |ir |2017-01-01|
|94         |0       |cg  |ir |2017-01-01|
|82         |0       |cg  |ir |2017-01-01|
|21         |0       |cg  |ir |2017-01-01|
|51         |0       |cg  |ir |2017-01-01|
|33         |0       |cg  |ir |2017-01-01|
|20         |0       |cg  |ir |2017-01-01|
|10         |0       |cg  |ir |2017-01-01|
|49         |0       |cg  |ir |2017-01-01|
|32         |0       |cg  |ir |2017-01-01|
|70         |0       |cg  |ir |2017-01-01|
|28         |0       |cg  |ir |2017-01-01|
|42         |0       |cg  |ir |2017-01-01|
|62         |0       |cg  |ir |2017-01-01|
|80         |0       |cg  |ir |2017-01-01|
|13         |0       |cg  |ir |2017-01-01|
|46         |0       |cg  |ir |2017-01-01|
|43         |0       |cg  |ir |2017-01-01|
|17         |0       |cg  |ir |2017-01-01|
|16         |0       |cg  |ir |2017-01-01|
+----------

# Explain Plan

In [15]:
query = """
SELECT
    date,
    MIN(date) OVER (ORDER BY date) AS base_date,
    FLOOR(months_between(date, (SELECT MIN(date) FROM flight))) AS month_index
FROM
    flight
ORDER BY 
    passengerID, 
    month_index
"""
spark.sql(query).show(10)
spark.sql(query).explain()

2022-02-22 16:00:39,899 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+----------+-----------+
|      date| base_date|month_index|
+----------+----------+-----------+
|2017-01-01|2017-01-01|          0|
|2017-11-29|2017-01-01|         10|
|2017-12-12|2017-01-01|         11|
|2017-12-22|2017-01-01|         11|
|2017-12-29|2017-01-01|         11|
|2017-01-01|2017-01-01|          0|
|2017-01-01|2017-01-01|          0|
|2017-01-10|2017-01-01|          0|
|2017-02-06|2017-01-01|          1|
|2017-03-05|2017-01-01|          2|
+----------+----------+-----------+
only showing top 10 rows

== Physical Plan ==
*(3) Project [date#26, base_date#85, month_index#87L]
+- *(3) Sort [passengerID#16 ASC NULLS FIRST, month_index#87L ASC NULLS FIRST], true, 0
   +- *(3) Project [date#26, base_date#85, month_index#87L, passengerID#16]
      +- Window [min(date#26) windowspecdefinition(date#26 ASC NULLS FIRST, specifiedwindowframe(RangeFrame, unboundedpreceding$(), currentrow$())) AS base_date#85], [date#26 ASC NULLS FIRST]
         +- *(2) Sort [date#26 ASC NULLS

2022-02-22 16:00:46,539 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [16]:
spark.sql(query).explain(True)

== Parsed Logical Plan ==
'Sort ['passengerID ASC NULLS FIRST, 'month_index ASC NULLS FIRST], true
+- 'Project ['date, 'MIN('date) windowspecdefinition('date ASC NULLS FIRST, unspecifiedframe$()) AS base_date#96, 'FLOOR('months_between('date, scalar-subquery#97 [])) AS month_index#98]
   :  +- 'Project [unresolvedalias('MIN('date), None)]
   :     +- 'UnresolvedRelation [flight], [], false
   +- 'UnresolvedRelation [flight], [], false

== Analyzed Logical Plan ==
date: date, base_date: date, month_index: bigint
Project [date#26, base_date#96, month_index#98L]
+- Sort [passengerID#16 ASC NULLS FIRST, month_index#98L ASC NULLS FIRST], true
   +- Project [date#26, base_date#96, month_index#98L, passengerID#16]
      +- Project [date#26, month_index#98L, base_date#96, base_date#96, passengerID#16]
         +- Window [min(date#26) windowspecdefinition(date#26 ASC NULLS FIRST, specifiedwindowframe(RangeFrame, unboundedpreceding$(), currentrow$())) AS base_date#96], [date#26 ASC NULLS FIRST]


2022-02-22 16:01:08,966 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


---
# Stop Spark Session

In [17]:
spark.stop()



# Cleanup

In [18]:
del spark
gc.collect()

273